## Архитектура MLflow

### Компонтенты MLflow и их задачи

 - MLflow Project - создание среды для экспериментов, группировка экспериментов.
 - MLflow Tracking - фиксация параметров и метрик качества экспериментов.
 - MLflow Models - подготовка версии модели к дистрибуции.
 - MLflow Registry - централизованное хранилище моделей и выкладка в эксплуатацию.

#### MLflow Projects

- Настройка окружения:
	- языки программирования
	- менеджер пакетов (например, conda)
	- зависимости (библиотеки xgboost, scikit-learn, ...)

- Описание окружения (Infrastructure as code):
	- различные ОС
	- локальное окружение
	- облачные сервисы

#### MLflow Tracking

Фиксирует все, что связано с запуском модели:
 - Наборы данных (для обучения и тестирования)
 - Наборы параметров (например, количество деревьев, слоев, L1/L2)
 - Значения метрик качества
 - Скорость работы и прочие технические метрики

#### MLflow Models

Cериализует артефакты моделей устраняя необходимость дополнительной разработки

#### MLflow Registry

 - Централизованное хранение версий моделей для удобного поиска.
 - Информация о том, какая модель используется на какой среде
 - История всех версий и их использования в средах



## 1. Работа с проектом и запуск эксперимента

Проект представляет собой папку с файлами, связанными с этим проектом:
 - Файл с метаданными MLProject (формат YAML) и также включаемые файлы (например, conda.yaml)
 - Файлы с кодом для запуска экспериментов (например, на Python)

Чтобы создать проект, дотсаточно описать корректный MLProject файл.

### Пример создания проекта

В файле проекта `MLproject` указано, что тренировка модели происходит в среде `conda`, и используется `scikit-learn` в качестве ML-библиотеки (указано в `conda.yaml`). Само обучение модели описано в файле `train.py`, там же описана необходимая подготовка данных.

In [1]:
%%writefile MLproject
name: tutorial

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"

Overwriting MLproject


In [2]:
%%writefile conda.yaml
name: tutorial
channels:
  - defaults
dependencies:
  - numpy>=1.14.3
  - pandas>=1.0.0
  - scikit-learn=0.19.1
  - pip
  - pip:
    - mlflow

Overwriting conda.yaml


Далее используется проект [Wine Quality](https://github.com/mlflow/mlflow-example)

#### Подготовка данных для эксперимента

Следующий код импортирует необходимые для работы модули, загружается данные из файла `wine-quality.csv` и разделяет их на тестовую и валидационную выборки

In [3]:
import os
import sys
import warnings
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [4]:
warnings.filterwarnings("ignore")
np.random.seed(40)
data = pd.read_csv("mlflow-example/wine-quality.csv")

train, test = train_test_split(data)

train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


### Создание и запуск эксперимента

Код самого эксперимента не зависит от MLflow, можно использовать уже готовый код.

Чтобы зафиксировать параметры запуска и метрики модели, нужно запустить обучение в рамках эксперимента и проекта.

`tracking_url` - адрес поднятого `mlflow` сервера, который будет использоваться для хранения экспериментов. Также по этому адресу доступен веб-интерфейс для просмотра результатов запусков.

you can run ml-flow server e.g. 

    mlflow server \
        --backend-store-uri sqlite:///mlflow.db \
        --default-artifact-root ./artifacts \
        --host 0.0.0.0

In [6]:
MLFLOW_SERVER_URL = 'http://127.0.0.1:5000/'  # 5000 is a default port

#### Запуск эксперимента

Чтобы запустить эксперимент, нужно выполнить код создания модели внутри контекста запуска MLflow и в этом контексте сохранить параметры и результирующие метрики.

In [7]:
# подключаемся к серверу
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'experiment2'
mlflow.set_experiment(experiment_name)

# запуск в эксперименте
with mlflow.start_run():
    alpha = 0.5
    l1_ratio = 0.5

    # модель
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # метрики
    predicted_qualities = lr.predict(test_x)
    rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))
    mae = mean_absolute_error(test_y, predicted_qualities)
    r2 = r2_score(test_y, predicted_qualities)

    print(f"Elasticnet model (alpha={alpha}, l1_ratio={l1_ratio}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

    # сохраняем значения эксперимента в системе
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.5, l1_ratio=0.5):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666


## 2. Подготовка модели к дистрибуции

Запуск успешного эксперимента можно подготовить к вводу в эксплуатацию.

Для этого используется  `MLflow Model Registry`.

### Просмотр проведенных экспериментов и выбор кандидата на ввод в эксплуатацию

Чтобы получить информацию о запусках экспериментов, нужно создать клиент `mlflow.tracking.MlflowClient`, затем выбрать нужный эксперимент и выбрать искомый запуск эксперимента.

В коде ниже берется последний запуск эксперимента из списка всех запусков.

In [8]:
client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)
run_info = client.list_run_infos(experiment.experiment_id)[-1]

print(experiment)
print(run_info)

<Experiment: artifact_location='./artifacts/1', experiment_id='1', lifecycle_stage='active', name='experiment2', tags={}>
<RunInfo: artifact_uri='./artifacts/1/e4cf109690bd40df8de6f4940b535cc5/artifacts', end_time=1650585325321, experiment_id='1', lifecycle_stage='active', run_id='e4cf109690bd40df8de6f4940b535cc5', run_uuid='e4cf109690bd40df8de6f4940b535cc5', start_time=1650585324048, status='FINISHED', user_id='sanityseeker'>


### Регистрация модели в MLflow Model Registry

Регистрация модели также доступна в веб-интерфейсе. Для этого нужно выбрать модель на странице экспериментов и нажать `Register Model`.

Ниже приведен код, совершающий аналогичное действие.

In [9]:
reg_model_name = "sk-learn-new-model"

# региструем модель
client.create_registered_model(reg_model_name)
# создаем первую версию
result = client.create_model_version(
    name=reg_model_name,
    source=f"{run_info.artifact_uri}/model",
    run_id=run_info.run_id
)

print(result)

RestException: RESOURCE_ALREADY_EXISTS: Registered Model (name=sk-learn-new-model) already exists. Error: (sqlite3.IntegrityError) UNIQUE constraint failed: registered_models.name
[SQL: INSERT INTO registered_models (name, creation_time, last_updated_time, description) VALUES (?, ?, ?, ?)]
[parameters: ('sk-learn-new-model', 1650747679970, 1650747679970, '')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

Посмотреть список зарегистрированных моделей можно следующим образом:

In [10]:
for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)

{   'creation_timestamp': 1650585329149,
    'description': '',
    'last_updated_timestamp': 1650585489590,
    'latest_versions': [   <ModelVersion: creation_timestamp=1650585329167, current_stage='Staging', description='', last_updated_timestamp=1650585489590, name='sk-learn-new-model', run_id='e4cf109690bd40df8de6f4940b535cc5', run_link='', source='./artifacts/1/e4cf109690bd40df8de6f4940b535cc5/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': 'sk-learn-new-model',
    'tags': {}}


Найти нужную модель и версию можно с помощью специальной функции:

In [11]:
for mv in client.search_model_versions(f"name='{reg_model_name}'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1650585329167,
    'current_stage': 'Staging',
    'description': '',
    'last_updated_timestamp': 1650585489590,
    'name': 'sk-learn-new-model',
    'run_id': 'e4cf109690bd40df8de6f4940b535cc5',
    'run_link': '',
    'source': './artifacts/1/e4cf109690bd40df8de6f4940b535cc5/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


## 3. Выкладка модели и тестирование сервера

Чтобы использовать модель в определенной среде, нужно перевести зарегистрирую модель в желаемую среду. Эта операция только регистрирует модель в желаемой среде, не запускает веб-сервер.

### Выкладка модели в тестовую и эксплуатационную среды

Терминология, использующаяся в MLflow Model Registry:

 - Staging - тестовая среда
 - Production - эксплуатационная среда
 
Следующий код переводит ранее зарегистрированную модель версии `1` в тестовую среду (`Staging`).

In [12]:
client.transition_model_version_stage(
    name=reg_model_name,
    version=1,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1650585329167, current_stage='Staging', description='', last_updated_timestamp=1650747954164, name='sk-learn-new-model', run_id='e4cf109690bd40df8de6f4940b535cc5', run_link='', source='./artifacts/1/e4cf109690bd40df8de6f4940b535cc5/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

Проверить результат можно следующим кодом:

In [19]:
for mv in client.search_model_versions(f"name='{reg_model_name}'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1650585329167,
    'current_stage': 'Staging',
    'description': '',
    'last_updated_timestamp': 1650747954164,
    'name': 'sk-learn-new-model',
    'run_id': 'e4cf109690bd40df8de6f4940b535cc5',
    'run_link': '',
    'source': './artifacts/1/e4cf109690bd40df8de6f4940b535cc5/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


Чтобы использовать модель, нужно запустить веб-сервер, в качестве параметров передать ему название модели и среды, а также порт (в примере запускается на 5005 порту).

Для взаимодействия с веб-сервером следует отправлять запросы к точке входа `/invocations` этого веб-сервера.

In [14]:
os.system('MLFLOW_TRACKING_URI=http://127.0.0.1:5000/ mlflow models serve -m "models:/sk-learn-new-model/Staging" -p 5005 --no-conda &')

0

/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py:2322: FutureWarning: `--no-conda` is deprecated and will be removed in a future MLflow release. Use `--env-manager=local` instead.
  value = self.callback(ctx, self, value)
2022/04/24 00:06:00 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/04/24 00:06:00 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5005 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-04-24 00:06:00 +0300] [515002] [INFO] Starting gunicorn 20.1.0
[2022-04-24 00:06:00 +0300] [515002] [INFO] Listening at: http://127.0.0.1:5005 (515002)
[2022-04-24 00:06:00 +0300] [515002] [INFO] Using worker: sync
[2022-04-24 00:06:00 +0300] [515004] [INFO] Booting worker with pid: 515004


In [15]:
import requests

url = 'http://127.0.0.1:5005/invocations'

http_data = test_x[:10].to_json(orient='split')
response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')

Predictions: [5.88527611335971, 5.740523335804658, 5.811935039818761, 5.957850132893952, 6.092739574397946, 5.38176881817212, 6.050858460676247, 5.992469597297115, 5.850426199303676, 5.627508796953737]


## 4. Задание для самосотоятельной работы

Задача: провести эксперимент и ввести в эксплуатацию новую обученную модель. 

1. С помощью MLflow Tracking в существующем проекте провести новый эксперимент с параметрами `alpha = 0.65`  `l1_ratio = 0.45`. 
    1. Зафиксировать результаты в системе, используя контекст `mlflow.start_run` и сохраняя параметры и метрики, вызвав `mlflow.log_param` и `mlflow.log_metric` соответственно. 

In [20]:
experiment_name = 'experiment3'
mlflow.set_experiment(experiment_name)

# запуск в эксперименте
with mlflow.start_run():
    alpha = 0.65
    l1_ratio = 0.45

    # модель
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # метрики
    predicted_qualities = lr.predict(test_x)
    rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))
    mae = mean_absolute_error(test_y, predicted_qualities)
    r2 = r2_score(test_y, predicted_qualities)

    print(f"Elasticnet model (alpha={alpha}, l1_ratio={l1_ratio}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

    # сохраняем значения эксперимента в системе
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

2022/04/24 01:57:30 INFO mlflow.tracking.fluent: Experiment with name 'experiment3' does not exist. Creating a new experiment.


Elasticnet model (alpha=0.65, l1_ratio=0.45):
  RMSE: 0.8337124532776438
  MAE: 0.6328980414913562
  R2: 0.10225612734318212


2. С помощью MLflow Model Registry зарегистрируйте проведенный эсперимент как модель для ввода в эксплуатацию. 
    1. Используя клиент `mlflow.tracking.MlflowClient` (переменная `client`) найдите проведенный эксперимент (например, по имени с помощью `client.get_experiment_by_name(experiment_name)`)
    2. Получите информацию о последнем запуске с помощью `client.list_run_infos`

    3. С помощью MLflow Model Registry создайте новую версию из зарегистрированной модели. 
      - Для этого с помощью `client.create_registered_model` зарегистрируйте модель под именем `coursera-top-model`, затем с помощью `client.create_model_version` создайте первую версию, указав также идентификатор запуска эксперимента из `run_info` и путь к модели, который также доступен в `run_info`.

In [21]:
client.get_experiment_by_name(experiment_name)

<Experiment: artifact_location='./artifacts/2', experiment_id='2', lifecycle_stage='active', name='experiment3', tags={}>

In [25]:
client.list_run_infos('2')

[<RunInfo: artifact_uri='./artifacts/2/4ae71dcc90f441948c444bdce4682a7b/artifacts', end_time=1650754651675, experiment_id='2', lifecycle_stage='active', run_id='4ae71dcc90f441948c444bdce4682a7b', run_uuid='4ae71dcc90f441948c444bdce4682a7b', start_time=1650754650872, status='FINISHED', user_id='sanityseeker'>]

In [34]:
task_run_id = client.list_run_infos('2')[0].run_id
task_artifact = client.list_run_infos('2')[0].artifact_uri

In [37]:
task_reg_name = 'coursera-top-model'

# регистрируем модель

# client.delete_registered_model(task_reg_name)
client.create_registered_model(task_reg_name)

result = client.create_model_version(
    name=task_reg_name,
    source=f"{task_artifact}/model",
    run_id=task_run_id
)

print(result)

2022/04/24 02:03:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: coursera-top-model, version 1


<ModelVersion: creation_timestamp=1650755038763, current_stage='None', description='', last_updated_timestamp=1650755038763, name='coursera-top-model', run_id='4ae71dcc90f441948c444bdce4682a7b', run_link='', source='./artifacts/2/4ae71dcc90f441948c444bdce4682a7b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


3. С помощью MLflow Model Registry выложите модель на тестовый сервер и проверьте, что API сервера возвращает результат. 
    1. Для этого с помощью `client.transition_model_version_stage` укажите имя зарегистрированной модели, версию и среду (`stage`). 
    2. Затем запустите сервер с моделью, используя консольный интерфейс `mlflow`: `! mlflow models serve -m "models:/имя_зарегистрированной_модели/Staging" -p 5005`. 
    3. Убедитесь, что сервер работает с помощью HTTP POST-запроса, используя библиотеку `requests`. Для этого возьмите несколько объектов из тестовой выборки и сформируйте запрос, сконвертировав данные в формат JSON: `test_x[:10].to_json(orient='split')`. Затем отправьте POST-запрос, указав в заголовке формат JSON: `requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)`, где `url` - адрес сервера с моделью (не адрес сервера MLFlow), например `http://127.0.0.1:5005/invocations`.

In [39]:
client.transition_model_version_stage(task_reg_name, 1, 'Staging')

<ModelVersion: creation_timestamp=1650755038763, current_stage='Staging', description='', last_updated_timestamp=1650755135525, name='coursera-top-model', run_id='4ae71dcc90f441948c444bdce4682a7b', run_link='', source='./artifacts/2/4ae71dcc90f441948c444bdce4682a7b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [ ]:
os.system('MLFLOW_TRACKING_URI=http://127.0.0.1:5000/ mlflow models serve -m "models:/coursera-top-model/Staging" -p 5005 --no-conda &')

In [42]:
import requests

url = 'http://127.0.0.1:5005/invocations'

http_data = test_x[:10].to_json(orient='split')
response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')

Predictions: [5.912666549805538, 5.8112236609337415, 5.8219284884820155, 5.97017317472579, 6.093062157940845, 5.3918182533300145, 5.986846332115556, 5.940343669181612, 5.889171748684333, 5.647494269772486]


4. Проведите аналогичные пункту 3 действия для эксплуатационного сервера.

In [43]:
client.transition_model_version_stage(task_reg_name, '1', 'Production')

<ModelVersion: creation_timestamp=1650755038763, current_stage='Production', description='', last_updated_timestamp=1650755379810, name='coursera-top-model', run_id='4ae71dcc90f441948c444bdce4682a7b', run_link='', source='./artifacts/2/4ae71dcc90f441948c444bdce4682a7b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [44]:
os.system('MLFLOW_TRACKING_URI=http://127.0.0.1:5000/ mlflow models serve -m "models:/coursera-top-model/Production" -p 5003 --no-conda &')

0

/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py:2322: FutureWarning: `--no-conda` is deprecated and will be removed in a future MLflow release. Use `--env-manager=local` instead.
  value = self.callback(ctx, self, value)
2022/04/24 02:09:59 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/04/24 02:09:59 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5003 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-04-24 02:09:59 +0300] [520032] [INFO] Starting gunicorn 20.1.0
[2022-04-24 02:09:59 +0300] [520032] [INFO] Listening at: http://127.0.0.1:5003 (520032)
[2022-04-24 02:09:59 +0300] [520032] [INFO] Using worker: sync
[2022-04-24 02:09:59 +0300] [520034] [INFO] Booting worker with pid: 520034


In [45]:
url = 'http://127.0.0.1:5003/invocations'
http_data = test_x[:10].to_json(orient='split')
response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')

Predictions: [5.912666549805538, 5.8112236609337415, 5.8219284884820155, 5.97017317472579, 6.093062157940845, 5.3918182533300145, 5.986846332115556, 5.940343669181612, 5.889171748684333, 5.647494269772486]
